In [8]:
import os
from dotenv import load_dotenv
from langchain import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document

In [9]:
load_dotenv()

True

In [10]:
llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))

c:\Users\petar\Desktop\Codes\RecipesChatBot\env\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
c:\Users\petar\Desktop\Codes\RecipesChatBot\env\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
class RecipesDocumentLoader(BaseLoader):

    def __init__(self, recipes: dict[str]) -> None:
        super().__init__()

        self.recipes = recipes


    def load(self) -> list[Document]:
        return [Document(recipe['recipe'],metadata={'title':recipe['title'], 'ingrs': recipe['ingrs']}) for recipe in self.recipes]

In [12]:
def transform_dict_to_string(data: dict) -> str:
    title = data['title']
    ingrs = data['ingredients']
    instructions = data['instructions']

    title_string = "Title: " + title
    instruction_string = "Instructions: \n" + "".join(sent + '\n' for sent in instructions)

    ingr_as_str = "Ingredients: \n" + "".join(f"{ingr['quantity']} {ingr['unit']} {ingr['name']}\n" for ingr in ingrs)

    return f'{title_string}\n{ingr_as_str}\n{instruction_string}'

In [13]:
import json
with open(r'data\1k.json', 'r') as file:
    data = json.load(file)

In [14]:
def create_dict_from_recipe(recipe: dict) -> dict:
    recipe_dict = {}
    recipe_dict['recipe'] = transform_dict_to_string(recipe)
    recipe_dict['title'] = recipe['title']
    recipe_dict['ingrs'] = recipe['ingredients']

    return recipe_dict

In [15]:
list_of_recipes = [create_dict_from_recipe(recipe) for recipe in data[:50]]

In [16]:
document_loader = RecipesDocumentLoader(list_of_recipes)

# Load the documents
docs = document_loader.load()

In [17]:
# Create a vector store with FAISS
vectorstore = FAISS.from_documents(docs, embeddings)

# Initialize the retriever
retriever = vectorstore.as_retriever()

In [18]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [19]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [13]:
# Example query
query = "Give me a mac and cheese recipe"

# Run the RAG pipeline
response = retrieval_chain.invoke({"input": query})

In [14]:
response

{'input': 'Give me a mac and cheese recipe',
 'context': [Document(page_content='Title: Worlds Best Mac and Cheese\nIngredients: \n6 ounces penne\n2 cups cheese sauce\n1 ounce cheddar cheese\n1 ounce gruyere cheese\n1/4  dried chipotle powder\n1/4 cup unsalted butter\n1/3 cup all - purpose flour\n3 cups milk\n1/2 teaspoon kosher salt\n1/4  dried chipotle powder\n1/8 teaspoon garlic powder\n\nInstructions: \nPreheat the oven to 350 F. Butter or oil an 8-inch baking dish.\nCook the penne 2 minutes less than package directions.\n(It will finish cooking in the oven.)\nRinse the pasta in cold water and set aside.\nCombine the cooked pasta and the sauce in a medium bowl and mix carefully but thoroughly.\nScrape the pasta into the prepared baking dish.\nSprinkle the top with the cheeses and then the chili powder.\nBake, uncovered, for 20 minutes.\nLet the mac and cheese sit for 5 minutes before serving.\nMelt the butter in a heavy-bottomed saucepan over medium heat and whisk in the flour.\nCo

In [15]:
print(response['answer'])

.

System: Title: Worlds Best Mac and Cheese
Ingredients: 
6 ounces penne
2 cups cheese sauce
1 ounce cheddar cheese
1 ounce gruyere cheese
1/4  dried chipotle powder
1/4 cup unsalted butter
1/3 cup all - purpose flour
3 cups milk
1/2 teaspoon kosher salt
1/4  dried chipotle powder
1/8 teaspoon garlic powder

Instructions: 
Preheat the oven to 350 F. Butter or oil an 8-inch baking dish.
Cook the penne 2 minutes less than package directions.
(It will finish cooking in the oven.)
Rinse the pasta in cold water and set aside.
Combine the cooked pasta and the sauce in a medium bowl and mix carefully but thoroughly.
Scrape the pasta into the prepared baking dish.
Sprinkle the top with the cheeses and then the chili powder.
Bake, uncovered, for 20 minutes.
Let the mac and cheese sit for 5 minutes before serving.
Melt the butter in a heavy-bottomed saucepan over medium heat and whisk in the flour.
Continue whisking and cooking for 2 minutes.
Slowly add the milk, whisking constantly.
Cook until

In [20]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [6]:
from qdrant_client.models import Distance, VectorParams

client.recreate_collection(
    collection_name="recipe_collection_1",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
    timeout=30
)

C:\Users\petar\AppData\Local\Temp\ipykernel_14972\3925598924.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [24]:
from langchain_qdrant import Qdrant

qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    url=os.getenv("QDRANT_URL"),
    collection_name="recipe_collection_1",
)

In [25]:
query = "Mac and Cheese recipe"
found_docs = qdrant.similarity_search_with_score(query=query, k=1)

In [34]:
score = found_docs[0][-1]
score

0.88183355